CSE 158/258 - Recommender Sys & Web Mining - FA24
# Assignment 2 Report

Andrew Choi (A69033628), Chanbin Na (AA18087468), Jonghee Chun (A69033997), Kenny Hwang (A99021639)

[Github Repo](https://github.com/cjychoi/cse258-fa24/tree/main)

## 1. Dataset and Exploratory Analysis